# Mutual Fund Investment Analysis

## Python (Flask) App, majorly for Backend APIs for Individual Mutual Fund Investment Analysis

### Problem Statement

Over the years, investors have tried and tested various methodologies to keep a track of all investements. The various problems faced for the same are -

1. Multi-vendor - The Mutual Funds are invested via multiple vendors, like Groww, Paytm Money, etc., hence no unified interface to track all investments.
2. The historic data is not accurately predicted - Over time the historic data loses importance and is overriden to be fitted only in Time Series Graph.
3. Periodic Tracking of data - Everyday net change in the invested amount.
4. Personalized Prediction - Currently all Mutual Fund Predictions are not personalized, only based on overall NAV changes.

### Data collection, Storage and Analysis Blueprint

Data from differen apps will be collected in the following way -

1. Per day data of return is manually entered in Google Sheet.
2. The data from the Google Sheet is fetched in Python, and stored in a MySQL DB.
3. Data fetching happens everyday, at 11:00 a.m. (APScheduler)
4. Success or failure mails for everyday update is triggered based on the storage of data in the respective database.
5. APIs expose the various data, grouped by various factors to be used in the UI.
6. Once the stored data crosses a significant volume, this data is splitted into train and test data for future analysis.
7. The predicted data is again exposed over APIs, grouped by various factors.